# TimesNet Training on ETT Datasets

This notebook trains TimesNet on all 4 ETT datasets (ETTh1, ETTh2, ETTm1, ETTm2) with different prediction horizons.

**Datasets:**
- ETTh1, ETTh2: Hourly data
- ETTm1, ETTm2: 15-minute data

**Configuration:**
- Input length: 96 (fixed, as in paper)
- Prediction horizons: {24, 48, 96, 192, 336, 720}
- Total experiments: 4 datasets × 6 horizons = 24 models

## Setup

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Add src to path
sys.path.append('./src')

from train import TimesNetTrainer
from evaluate import evaluate_and_save_results, plot_training_curves

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")

## Configuration

In [ ]:
# Base paths
ROOT_PATH = './ETDataset/ETT-small/'
CHECKPOINT_BASE = './checkpoints'
RESULTS_DIR = './results'

# Create directories
os.makedirs(CHECKPOINT_BASE, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Datasets
DATASETS = ['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2']

# Fixed input length (as in paper)
SEQ_LEN = 96

# Prediction horizons
PRED_LENS = [24, 48, 96, 192, 336, 720]

# Model hyperparameters (from paper)
MODEL_CONFIGS = {
    'ETTh1': {'d_model': 16, 'd_ff': 32},
    'ETTh2': {'d_model': 16, 'd_ff': 32},
    'ETTm1': {'d_model': 32, 'd_ff': 64},
    'ETTm2': {'d_model': 32, 'd_ff': 64},
}

# Training config
TRAIN_CONFIG = {
    'enc_in': 7,
    'c_out': 7,
    'top_k': 5,
    'e_layers': 2,
    'num_kernels': 6,
    'dropout': 0.1,
    'embed': 'fixed',
    'batch_size': 32,
    'learning_rate': 0.0001,
    'train_epochs': 10,
    'patience': 3,
    'lradj': 'type1',
    'use_amp': False,
    'num_workers': 0,
}

print(f"✓ Configuration set")
print(f"  - Datasets: {DATASETS}")
print(f"  - Input length: {SEQ_LEN}")
print(f"  - Prediction horizons: {PRED_LENS}")
print(f"  - Total experiments: {len(DATASETS) * len(PRED_LENS)}")

## Training Function

In [ ]:
def train_single_model(dataset_name, pred_len):
    """
    Train a single TimesNet model
    
    Args:
        dataset_name: Dataset name (e.g., 'ETTh1')
        pred_len: Prediction horizon
    
    Returns:
        dict: Training results
    """
    print("\n" + "="*70)
    print(f"Training: {dataset_name} | seq_len={SEQ_LEN} → pred_len={pred_len}")
    print("="*70)
    
    # Create config
    config = {
        'root_path': ROOT_PATH,
        'data_path': f'{dataset_name}.csv',
        'seq_len': SEQ_LEN,
        'pred_len': pred_len,
        'checkpoints': f'{CHECKPOINT_BASE}/{dataset_name}_{SEQ_LEN}_{pred_len}',
        **TRAIN_CONFIG,
        **MODEL_CONFIGS[dataset_name]
    }
    
    # Create checkpoint directory
    os.makedirs(config['checkpoints'], exist_ok=True)
    
    # Train
    trainer = TimesNetTrainer(config)
    train_losses, val_losses = trainer.train()
    
    # Test
    test_results = trainer.test()
    
    # Save training curves
    curve_path = f'{RESULTS_DIR}/{dataset_name}_{SEQ_LEN}_{pred_len}_curves.png'
    plot_training_curves(train_losses, val_losses, save_path=curve_path)
    
    # Prepare results
    results = {
        'dataset': dataset_name,
        'seq_len': SEQ_LEN,
        'pred_len': pred_len,
        'd_model': config['d_model'],
        'd_ff': config['d_ff'],
        'train_losses': train_losses,
        'val_losses': val_losses,
        'test_mse': test_results['mse'],
        'test_mae': test_results['mae'],
        'test_rmse': test_results['rmse'],
        'final_epoch': len(train_losses),
    }
    
    print(f"\n✓ Completed: {dataset_name}_{SEQ_LEN}_{pred_len}")
    print(f"  - Test MSE: {test_results['mse']:.6f}")
    print(f"  - Test MAE: {test_results['mae']:.6f}")
    print(f"  - Test RMSE: {test_results['rmse']:.6f}")
    
    return results

## Train All Models

In [ ]:
# Store all results
all_results = []

# Train all combinations
total_experiments = len(DATASETS) * len(PRED_LENS)
current_experiment = 0

for dataset_name in DATASETS:
    for pred_len in PRED_LENS:
        current_experiment += 1
        
        print(f"\n{'#'*70}")
        print(f"# Experiment {current_experiment}/{total_experiments}")
        print(f"{'#'*70}")
        
        try:
            results = train_single_model(dataset_name, pred_len)
            all_results.append(results)
            
        except Exception as e:
            print(f"\n❌ Error training {dataset_name}_{SEQ_LEN}_{pred_len}: {e}")
            import traceback
            traceback.print_exc()
            continue

print("\n" + "="*70)
print("✓ ALL TRAINING COMPLETED!")
print("="*70)

## Save Results

In [ ]:
# Convert to DataFrame for easy analysis
results_df = pd.DataFrame([{
    'dataset': r['dataset'],
    'seq_len': r['seq_len'],
    'pred_len': r['pred_len'],
    'd_model': r['d_model'],
    'd_ff': r['d_ff'],
    'test_mse': r['test_mse'],
    'test_mae': r['test_mae'],
    'test_rmse': r['test_rmse'],
    'final_epoch': r['final_epoch']
} for r in all_results])

# Save to CSV
csv_path = f'{RESULTS_DIR}/all_results.csv'
results_df.to_csv(csv_path, index=False)
print(f"✓ Results saved to {csv_path}")

# Save detailed results (with training curves) to JSON
json_path = f'{RESULTS_DIR}/all_results_detailed.json'
with open(json_path, 'w') as f:
    json.dump(all_results, f, indent=2)
print(f"✓ Detailed results saved to {json_path}")

# Display summary
print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)
display(results_df)

## Results Analysis

### Results by Dataset

In [ ]:
# Group by dataset
for dataset in DATASETS:
    print(f"\n{'='*60}")
    print(f"{dataset} Results")
    print(f"{'='*60}")
    
    dataset_results = results_df[results_df['dataset'] == dataset]
    display(dataset_results[['pred_len', 'test_mse', 'test_mae', 'test_rmse', 'final_epoch']])

### Visualizations

In [ ]:
# Plot MSE vs Prediction Horizon for each dataset
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, dataset in enumerate(DATASETS):
    ax = axes[idx]
    dataset_results = results_df[results_df['dataset'] == dataset].sort_values('pred_len')
    
    ax.plot(dataset_results['pred_len'], dataset_results['test_mse'], 
            marker='o', linewidth=2, markersize=8, label='MSE')
    ax.plot(dataset_results['pred_len'], dataset_results['test_mae'], 
            marker='s', linewidth=2, markersize=8, label='MAE')
    
    ax.set_xlabel('Prediction Horizon', fontsize=12)
    ax.set_ylabel('Error', fontsize=12)
    ax.set_title(f'{dataset} - Error vs Prediction Horizon', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/error_vs_horizon.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Plot saved to {RESULTS_DIR}/error_vs_horizon.png")

In [ ]:
# Heatmap of MSE across datasets and horizons
pivot_mse = results_df.pivot(index='dataset', columns='pred_len', values='test_mse')

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_mse, annot=True, fmt='.4f', cmap='YlOrRd', 
            cbar_kws={'label': 'Test MSE'}, linewidths=0.5)
plt.title('Test MSE: Dataset vs Prediction Horizon', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Prediction Horizon', fontsize=12)
plt.ylabel('Dataset', fontsize=12)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/mse_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Heatmap saved to {RESULTS_DIR}/mse_heatmap.png")

In [ ]:
# Heatmap of MAE across datasets and horizons
pivot_mae = results_df.pivot(index='dataset', columns='pred_len', values='test_mae')

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_mae, annot=True, fmt='.4f', cmap='YlGnBu', 
            cbar_kws={'label': 'Test MAE'}, linewidths=0.5)
plt.title('Test MAE: Dataset vs Prediction Horizon', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Prediction Horizon', fontsize=12)
plt.ylabel('Dataset', fontsize=12)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/mae_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Heatmap saved to {RESULTS_DIR}/mae_heatmap.png")

### Statistical Summary

In [ ]:
# Summary statistics
print("="*70)
print("STATISTICAL SUMMARY")
print("="*70)

print("\nOverall Statistics:")
print(results_df[['test_mse', 'test_mae', 'test_rmse']].describe())

print("\nBest Results (by MSE):")
best_mse = results_df.loc[results_df.groupby('dataset')['test_mse'].idxmin()]
display(best_mse[['dataset', 'pred_len', 'test_mse', 'test_mae', 'test_rmse']])

print("\nBest Results (by MAE):")
best_mae = results_df.loc[results_df.groupby('dataset')['test_mae'].idxmin()]
display(best_mae[['dataset', 'pred_len', 'test_mse', 'test_mae', 'test_rmse']])

### Comparison with Paper Results

In [ ]:
# Paper results (from TimesNet paper Table 1)
paper_results = {
    'ETTh1': {
        96: {'mse': 0.384, 'mae': 0.402},
        192: {'mse': 0.436, 'mae': 0.429},
        336: {'mse': 0.491, 'mae': 0.469},
        720: {'mse': 0.521, 'mae': 0.491},
    },
    'ETTm1': {
        96: {'mse': 0.334, 'mae': 0.365},
        192: {'mse': 0.374, 'mae': 0.385},
        336: {'mse': 0.410, 'mae': 0.403},
        720: {'mse': 0.478, 'mae': 0.437},
    }
}

# Compare with our results
print("="*70)
print("COMPARISON WITH PAPER RESULTS")
print("="*70)

for dataset in ['ETTh1', 'ETTm1']:
    if dataset in paper_results:
        print(f"\n{dataset}:")
        print("-" * 60)
        print(f"{'Horizon':>10} {'Paper MSE':>12} {'Our MSE':>12} {'Diff':>10} {'Paper MAE':>12} {'Our MAE':>12} {'Diff':>10}")
        print("-" * 60)
        
        for horizon in [96, 192, 336, 720]:
            if horizon in paper_results[dataset]:
                paper = paper_results[dataset][horizon]
                our = results_df[(results_df['dataset'] == dataset) & (results_df['pred_len'] == horizon)]
                
                if not our.empty:
                    our_mse = our['test_mse'].values[0]
                    our_mae = our['test_mae'].values[0]
                    
                    mse_diff = ((our_mse - paper['mse']) / paper['mse']) * 100
                    mae_diff = ((our_mae - paper['mae']) / paper['mae']) * 100
                    
                    print(f"{horizon:>10} {paper['mse']:>12.4f} {our_mse:>12.4f} {mse_diff:>9.2f}% "
                          f"{paper['mae']:>12.4f} {our_mae:>12.4f} {mae_diff:>9.2f}%")

## Export Results Table (LaTeX)

In [ ]:
# Create LaTeX table
latex_table = []
latex_table.append("\\begin{table}[h]")
latex_table.append("\\centering")
latex_table.append("\\caption{TimesNet Results on ETT Datasets}")
latex_table.append("\\begin{tabular}{lcccc}")
latex_table.append("\\hline")
latex_table.append("Dataset & Horizon & MSE & MAE & RMSE \\\\")
latex_table.append("\\hline")

for dataset in DATASETS:
    dataset_results = results_df[results_df['dataset'] == dataset].sort_values('pred_len')
    for _, row in dataset_results.iterrows():
        latex_table.append(f"{row['dataset']} & {row['pred_len']} & "
                          f"{row['test_mse']:.4f} & {row['test_mae']:.4f} & {row['test_rmse']:.4f} \\\\")
    latex_table.append("\\hline")

latex_table.append("\\end{tabular}")
latex_table.append("\\end{table}")

latex_str = "\n".join(latex_table)

# Save to file
latex_path = f'{RESULTS_DIR}/results_table.tex'
with open(latex_path, 'w') as f:
    f.write(latex_str)

print(f"✓ LaTeX table saved to {latex_path}")
print("\nLaTeX Table:")
print(latex_str)

## Summary

✅ **Training Complete!**

**Trained Models:**
- 4 datasets (ETTh1, ETTh2, ETTm1, ETTm2)
- 6 prediction horizons each (24, 48, 96, 192, 336, 720)
- Total: 24 models

**Output Files:**
- `results/all_results.csv` - Summary table
- `results/all_results_detailed.json` - Detailed results with training curves
- `results/error_vs_horizon.png` - Error vs horizon plots
- `results/mse_heatmap.png` - MSE heatmap
- `results/mae_heatmap.png` - MAE heatmap
- `results/results_table.tex` - LaTeX table
- `results/{dataset}_{seq_len}_{pred_len}_curves.png` - Individual training curves
- `checkpoints/{dataset}_{seq_len}_{pred_len}/checkpoint.pth` - Model checkpoints